In [1]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 

c:\Users\ashle\Anaconda\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()



==
Logging in..
Dimcli - Dimensions API Client (v1.2)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.9
Method: manual login


In [3]:

#Link to grant variables description: https://docs.dimensions.ai/dsl/datasource-grants.html#grants
search_Var = "[abstract + active_year + category_bra + category_for + category_for_2020 + category_hra + category_hrcs_hc + category_hrcs_rac + category_icrp_cso + category_icrp_ct + category_rcdc + category_sdg + category_uoa + concepts + concepts_scores + date_inserted + dimensions_url + end_date + foa_number + funder_countries + funder_org_acronym + funder_org_cities + funder_org_countries + funder_org_name + funder_org_states + funder_orgs + funders + funding_aud + funding_cad + funding_chf + funding_cny + funding_currency + funding_eur + funding_gbp + funding_jpy + funding_nzd + funding_org_acronym + funding_org_cities + funding_org_city + funding_org_name + funding_org_states + funding_schemes + funding_usd + grant_number + id + investigators + keywords + language + language_title + linkout + original_title + project_numbers + research_org_cities + research_org_countries + research_org_names + research_org_state_codes + research_orgs + researchers + score + start_date + start_year + title]"
names = {}

#CHANGE CSV FILE TO THE NAME OF YOUR FILE SAVED FROM DimensionsAI_IDs.ipynb
with open('1995_2010.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader, None)  
    for row in reader:
        name = row[0]
        openIds_str = row[2]  
        openIds_list = ast.literal_eval(openIds_str)
        names[name] = openIds_list

print(names)


{"alan d'andrea": ['ur.012727024557.25'], 'james fagin': ['ur.01002446502.38'], 'daniel haber': ['ur.01221020161.61'], 'ethylin jabs': ['ur.0757041626.26'], 'karl nath': ['ur.0621613670.95'], 'bishr omary': ['ur.014730211054.29'], 'john raymond': ['ur.0717265610.95'], 'fuad ziyadeh': ['ur.0761676730.05'], 'maria abreu': ['ur.0606477677.70'], 'michael ackerman': ['ur.01227355747.25'], 'mariana kaplan': ['ur.012647507142.53'], 'feroz papa': ['ur.01147037443.33'], 'paul taylor': ['ur.013436160257.94'], 'loren walensky': ['ur.01324277771.74'], 'yiping yang': ['ur.0707211155.90'], 'steven albelda': ['ur.015000016117.95'], 'richard  anderson': ['ur.010576736217.64'], 'peter arvan': ['ur.01002760245.88'], 'adam asch': ['ur.0615715006.01'], 'john belmont': ['ur.01140262452.40'], 'bradford berk': ['ur.012523312277.63'], 'morris birnbaum': ['ur.01033446252.27'], 'richard blumberg': ['ur.0633257054.96'], 'douglas bradley': ['ur.01153467234.36'], 'kenneth bridges': ['ur.016162551417.02'], 'william

In [4]:
def findby_idGrant(nameDict):
    all_Grants= {}
    count=0
    for name, person_ids in nameDict.items():
        indv_grant_list = []
        for i in person_ids:
            res = dsl.query_iterative(f"""search grants where researchers = "{i}" return grants{search_Var}""")
            number = res.json['_stats']['total_count']
            print(f'{i} id has {number} grants')
            listDict = res.json.get('grants', [])

            indv_grant_list.extend(listDict)

        all_Grants[name] = indv_grant_list
        count+=1
        print(f'{count}. {name}')
        print(len(all_Grants[name]))
    return all_Grants

grants = findby_idGrant(names) 

json_filename = "all_Grants.json"

with open(json_filename, 'w') as json_file:
    json.dump(grants, json_file, indent=2)

print(f'The data has been written to {json_filename}')

Starting iteration with limit=1000 skip=0 ...
0-27 / 27 (0.65s)
===
Records extracted: 27
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.012727024557.25 id has 27 grants
1. alan d'andrea
27


0-18 / 18 (4.67s)
===
Records extracted: 18
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.01002446502.38 id has 18 grants
2. james fagin
18


0-22 / 22 (0.70s)
===
Records extracted: 22
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


...

ur.01024164315.00 id has 16 grants
158. thomas wang
16


0-13 / 13 (0.77s)
===
Records extracted: 13
Warnings:  8
Starting iteration with limit=1000 skip=0 ...


ur.0652523657.55 id has 13 grants
159. myles wolf
13


0-13 / 13 (4.42s)
===
Records extracted: 13
Warnings:  8


ur.015100501062.34 id has 13 grants
160. alan yu
13
The data has been written to all_Grants.json


In [2]:
import csv
import json

csv_filename = "dimensionsGrants_data.csv"

with open("all_Grants.json", 'r', encoding='utf-8') as file:
    personDict = json.load(file)
# removed 'investigators
common_work_info_headers = ['abstract', 'active_years', 'altmetric', 'associated_grant_ids', 'brief_title', 'category_bra', 'category_for', 'category_for_2020', 'category_hra', 'category_hrcs_rac', 'category_rcdc', 'conditions', 'date', 'date_inserted', 'dimensions_url', 'end_date', 'funder_countries', 'funders', 'funding_usd','grant_number','gender', 'id', 'interventions', 'linkout', 'mesh_terms', 'phase', 'publication_ids', 'publications', 'registry', 'score', 'start_date', 'study_arms', 'study_designs', 'study_eligibility_criteria', 'study_maximum_age', 'study_minimum_age', 'study_outcome_measures', 'study_participants', 'study_type', 'title']


with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    
    header = ['Person'] + common_work_info_headers
    writer.writerow(header)

    # Write data rows
    for person, works in personDict.items():
        for work in works:  # Assuming works is a list of dictionaries now
            row = [person] + [work.get(head, '') for head in common_work_info_headers]
            writer.writerow(row)

print(f"CSV file '{csv_filename}' created successfully.")

CSV file 'dimensionsGrants_data.csv' created successfully.
